In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text

     |████████████████████████████████| 4.3MB 8.2MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install -q tf-models-official

In [9]:
!pip3 install tensorflow_text

     |████████████████████████████████| 4.3MB 6.7MB/s 


In [10]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt


tf.get_logger().setLevel('ERROR')

In [11]:
!pip3 install ktrain

import pandas as pd
import numpy as np

import ktrain
from ktrain import text

     |████████████████████████████████| 25.3MB 121kB/s 
     |████████████████████████████████| 6.8MB 43.8MB/s 
     |████████████████████████████████| 983kB 45.7MB/s 
     |████████████████████████████████| 266kB 43.4MB/s 
     |████████████████████████████████| 1.9MB 42.2MB/s 
     |████████████████████████████████| 471kB 47.3MB/s 
     |████████████████████████████████| 3.3MB 45.8MB/s 
     |████████████████████████████████| 901kB 36.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.3-cp37-none-any.whl size=25282380 sha256=bdb74b34ad7c267fa240bf99cb8f8a958dacd7e6327aa6c2a6eac3c161a3c695
  Stored in directory: /root/.cache/pip/wheels/16/05/be/d6e659b3349016b1059e19fa028f165af4eeae2c196f329112
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993223 sha256=111b48bf6d8c15a8d0c0b043781d414b9c6a6dd7600ce37975551a85c822729c
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7ddc6c92d3aa015cf1a498df5a70268996
  Created wheel for syn

In [13]:
data_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp-text-emotion-master/nlp-text-emotion-master/data/data_train.csv', encoding='utf-8')
data_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp-text-emotion-master/nlp-text-emotion-master/data/data_test.csv', encoding='utf-8')

X_train = data_train.Text.tolist()
X_test = data_test.Text.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

data.head(10)

size of training set: 7934
size of validation set: 3393
joy        2326
sadness    2317
anger      2259
neutral    2254
fear       2171
Name: Emotion, dtype: int64


Emotion                                               Text
0  neutral   There are tons of other paintings that I thin...
1  sadness  Yet the dog had grown old and less capable , a...
2     fear  When I get into the tube or the train without ...
3     fear  This last may be a source of considerable disq...
4    anger  She disliked the intimacy he showed towards so...
5  sadness  When my family heard that my Mother's cousin w...
6      joy  Finding out I am chosen to collect norms for C...
7    anger  A spokesperson said : ` Glen is furious that t...
8  neutral                                             Yes . 
9  sadness  When I see people with burns I feel sad, actua...

In [14]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'neutral': 4
}

# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [15]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [16]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Is Multi-Label? False
maxlen is 350
done.


In [17]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [19]:
learner.fit_onecycle(1e-5, 3)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/3
1323/1323 [==============================] - 769s 569ms/step - loss: 0.3800 - accuracy: 0.5597 - val_loss: 0.2248 - val_accuracy: 0.7801
Epoch 2/3
1323/1323 [==============================] - 754s 570ms/step - loss: 0.1816 - accuracy: 0.8324 - val_loss: 0.1879 - val_accuracy: 0.8123
Epoch 3/3
1323/1323 [==============================] - 753s 569ms/step - loss: 0.1037 - accuracy: 0.9148 - val_loss: 0.1865 - val_accuracy: 0.8235


In [21]:
#learner.autofit(1e-4)

In [22]:
predictor=ktrain.get_predictor(learner.model,preproc)

In [24]:
predictor.predict('I am very happy')

[('joy', 0.99924964),
 ('sadness', 0.00013465018),
 ('fear', 9.964195e-05),
 ('anger', 0.00012885295),
 ('neutral', 0.00038720033)]

In [25]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 350)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 350)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 350, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 350, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [26]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['joy', 'sadness', 'fear', 'anger', 'neutral']

In [46]:
import time 

message = 'We got a new puppy'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))


predicted: [('joy', 0.926989), ('sadness', 0.0011700712), ('fear', 0.0014477153), ('anger', 0.0026255543), ('neutral', 0.0677677)] (0.07)


In [32]:
data = ['this movie was horrible, the plot was really boring. acting was okay',
        'the fild is really sucked. there is not plot and acting was bad',
        'what a beautiful movie. great plot. acting was good. will see it again']

In [33]:
predictor.save('/content/drive/My Drive/data/bert_final')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [34]:
!zip -r /content/bert.zip /content/bert

	zip warning: name not matched: /content/bert

zip error: Nothing to do! (try: zip -r /content/bert.zip . -i /content/bert)


In [35]:
learner.view_top_losses(n=1, preproc=preproc)

----------
id:2702 | loss:3.84 | true:[0. 0. 0. 0. 1.] | pred:[0. 0. 1. 0. 0.])

[CLS] i was skiing on and on excitedly until i realized one thing - i could not stop ! you can just imagine how terrified i was . [SEP]


In [36]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

/usr/local/lib/python3.7/dist-packages/ktrain/core.py:146: UserWarning: multilabel confusion matrices not yet supported
  warnings.warn('multilabel confusion matrices not yet supported')


In [37]:
print(X_test[2242])

 I am sorry to say it's not on the way , but during loading . The tins inside the cases were broken evidently through rough handling . 


In [38]:
predictor.predict(data, return_proba=True)

array([[9.3405100e-04, 9.7740608e-01, 8.8806413e-03, 7.8129675e-03,
        4.9662762e-03],
       [1.9254773e-03, 3.6444041e-01, 5.7175910e-01, 5.4447860e-02,
        7.4270908e-03],
       [9.9815422e-01, 2.3564526e-04, 1.2022144e-03, 9.2946757e-05,
        3.1490601e-04]], dtype=float32)

In [39]:
predictor.predict(data)

[[('joy', 0.000934051),
  ('sadness', 0.9774061),
  ('fear', 0.008880641),
  ('anger', 0.0078129675),
  ('neutral', 0.004966276)],
 [('joy', 0.0019254773),
  ('sadness', 0.3644404),
  ('fear', 0.5717591),
  ('anger', 0.05444786),
  ('neutral', 0.007427091)],
 [('joy', 0.9981542),
  ('sadness', 0.00023564526),
  ('fear', 0.0012022144),
  ('anger', 9.294676e-05),
  ('neutral', 0.000314906)]]